# Project: Investigate a Dataset (FBI Gun Data)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

## Introduction


#### What is the overall trend of gun purchases?
#### What states have the highest volume of gun registrations?
#### What states have the highest gun registrations per capita?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib  inline 

<a id='wrangling'></a>
## Data Wrangling

> load the data, check for cleanliness, and trim and clean the datset

In [ ]:
# load the data
df_guns = pd.read_excel('data/gun-data.xlsx')

df_us_census = pd.read_csv('data/U.S. Census Data.csv', sep =',')

 ####  FBI Guns Data

In [ ]:
df_guns.head(10)

In [ ]:
df_guns.shape # check the characteristics (27) and the number of observations (12485)

In [ ]:
df_guns.info()  # check the summary of the dataframe,
                # including the number of columns and rows, data types, and number of non-null values in each column

In [ ]:
df_guns.isnull().sum()  # check for missing value count for each column 

In [ ]:
df_guns.describe() # check summary statistics 

### Census Data

In [ ]:
df_us_census.head(5)

In [ ]:
df_us_census.info() 

## Data Cleaning  

trim and clean the datasets

### FBI Guns Data

After assessing the guns data, I'm going to convert month to datetime.

In [ ]:
#timestamps are represented as strings instead of datetime 
df_guns['month'] = pd.to_datetime(df_guns.month, format= "%Y-%m")

df_guns.month # check for correct month data type after change

The number of unique values in the state column shows several states/territories that are inconsistent with the US Census data.

In [ ]:
df_us_census.info()

I'm going to filter and drop the states/territories that are missing from the US Census Data.

In [ ]:
guns = pd.Series(df_guns['state'].unique())
census = pd.Series(df_us_census.columns[2:])

missing_in_census = guns[~guns.isin(census)]

missing_in_census

In [ ]:
df_guns.drop = df_guns.query("state != ('Guam','Mariana Islands', 'Puerto Rico', 'Virgin Islands', 'District of Columbia')", inplace=True)

df_guns['state'].unique()

In [ ]:
# make state lowercase for readability 
df_guns["state"] = df_guns["state"].str.lower() 

https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset

check the percentage of missing data in the guns dataset for every column

In [ ]:
#calculate the total percentage of null values in every column of the FBI guns data
percent_missing = df_guns.isnull().sum() * 100 / len(df_guns)
missing_values = pd.DataFrame({'column_name': df_guns.columns,
                                 'percent_missing': percent_missing})

missing_values #the returned, rentals, and private gun types have a high percentage of missing date

In [ ]:
df_guns.fillna(0,inplace=True) #filling nulls with 0

In [ ]:
df_guns.info() # confirm correction with info()

https://stackoverflow.com/questions/45985877/slicing-multiple-column-ranges-from-a-dataframe-using-iloc

In [ ]:
# display a histogram of - handgun, long_gun, and totals - all seem to be skewed to the right with long guns having a slightly higher number of observations in different bins

df_guns.iloc[:,np.r_[4:6,26]].hist(figsize=(10,8));

##### Data Cleaning: Census Data

join the US Census data with the FBI Guns data

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transpose.html

https://knowledge.udacity.com/questions/428050

In [ ]:
df_us_census.T.head(10) #transpose the df to swap rows and columns

In [ ]:
df_us_census['Fact'].unique()  # check unique rows in the fact column

In [ ]:
df_us_census['Fact Note'].unique() # check unique rows in the fact note column

create a new df and drop unncessary columns 

In [ ]:
#  drop columns that aren't useful since we swapped fact rows to columns
df_us_census_2 = df_us_census.T
df_us_census_2.columns = df_us_census_2.loc['Fact']
df_us_census_2.drop(['Fact','Fact Note'],inplace=True)

df_us_census_2.head(5)

#### Remove all columns between column index 3 to 86 since we're only going to look at 2016 and 2010 census population data

In [ ]:
df_us_census_2.drop(df_us_census_2.iloc[:, 2:86], inplace = True, axis = 1)

In [ ]:
df_us_census_2.info()

In [ ]:
df_us_census_2.rename(columns={"Population estimates, July 1, 2016,  (V2016)": "2016_Population_Estimates", "Population estimates base, April 1, 2010,  (V2016)": "2010_Population_Estimates"}, inplace=True)


df_us_census_2

I'm going to replace commas and missing spaces with underscores

In [ ]:
# to join with FBI guns replace commas and missing spaces and underscores 
df_us_census_2.columns = [str(x).replace(',','_').replace(' ','_') for x in df_us_census_2.columns]



In [ ]:
df_us_census_2.head()

https://knowledge.udacity.com/questions/428050

In [ ]:
df_us_census_2.info() # check index to drop columns

https://github.com/malaklm/solution/blob/master/US%20Census%20data.ipynb

Replace commas in dataset to convert to floats 

In [ ]:
df_us_census_2.replace({",": ''}, regex=True,inplace=True)
df_us_census_2.info()

Convert strings to floats in order to perform calculations

In [ ]:
df_us_census_2['2016_Population_Estimates'] = pd.to_numeric(df_us_census_2['2016_Population_Estimates'],errors='coerce').astype(float)
df_us_census_2['2010_Population_Estimates'] = pd.to_numeric(df_us_census_2['2010_Population_Estimates'],errors='coerce').astype(float) 

df_us_census_2.info() # check if data type conversion worked

In [ ]:
df_us_census_2.head(5)

<a id='eda'></a>
## Exploratory Data Analysis


What is the overall trend of gun registrations?

What state has the highest volume of gun registrations?

What state has the highest guns per capita?


### Research Question 1: What is the overall trend of gun purchases?

Analyze average count of permits for a single variables in dataset pertaining to gun types 

https://towardsdatascience.com/build-the-story-around-data-using-exploratory-data-analysis-and-pandas-c85bf3beff87

In [ ]:
# here is a statistic for understanding the distribution of gun registrations 

avg_handgun = df_guns['handgun'].mean() # calculate the avg number of handgun registrations across the dataframe

# check frequency and distribution of hand gun registrations with a histogram
plt.hist(df_guns.handgun)
plt.plot()
plt.title("Histogram of Hand Guns",fontsize=15)
plt.ylabel('Frequency of Gun Registrations', fontsize=12)
plt.xlabel('Hand Guns', fontsize=12)
plt.legend

print(f'{avg_handgun:0,.2f}, is the average number of gun permits for handguns nationwide.')


print('''The histogram example below represents gun registrations for handguns. 
Essentially, distribtion is positively skewed right with registrations most frequently in the 10K range, followed by 1K.''')

In [ ]:
# What is the avg count of permits for long guns? calculate the mean of long guns to display average
avg_long_gun = df_guns['long_gun'].mean()


# check frequency and distribution of long gun registrations with a histogram
plt.hist(df_guns.long_gun)
plt.plot()
plt.title("Histogram of Long Guns",fontsize=15)
plt.ylabel('Frequency of Gun Registrations', fontsize=12)
plt.xlabel('Long Guns', fontsize=12)
plt.legend


print(f"Here is the average number of long_gun registrations nationwide: {avg_long_gun:0,.2f}.")
print('''Consistent with handguns, long guns have similar distribution skewed to the right. 
Long guns had frequency of registrations fall within the 8k and 2k range with the number of observations mostly in the 8k bin''')

In [ ]:
# What is the avg count of permits for multiple gun gytpes? calculate the mean of multiple gun types to display average
avg_multiple_gun = df_guns['multiple'].mean()


# check frequency and distribution of multiple registrations with a histogram
plt.hist(df_guns.multiple)
plt.plot()
plt.title("Histogram of Multiple Guns in FBI Data",fontsize=15)
plt.ylabel('Frequency of Gun Registrations', fontsize=12)
plt.xlabel('Multiple Guns', fontsize=12)
plt.legend

print(f"Here is the average count of gun registrations for multiple gun types:, {avg_multiple_gun:0,.2f}")
print('''The histogram shows that the data is exclsuively skewed to the right with frequency 
of registrations around 11K''')

### Analyze Overall Purchases in the Dataset using month and totals

https://seaborn.pydata.org/examples/timeseries_facets.html

https://stackoverflow.com/questions/65300109/generating-a-line-graph-using-seaborn-or-matplotlib-with-year-as-hue-month-as

In [ ]:
df_guns_q1 = df_guns #making a new df to analyze overall FBI gun permit totals in the nation

In [ ]:
df_guns_q1 = df_guns_q1.reset_index(drop=True) # reset the index in case indices are out of order

In [ ]:
# drop columns with high volume of nulls and to simplify analysis
df_guns_q1.drop(['permit','permit_recheck','other','admin','prepawn_handgun','prepawn_long_gun','prepawn_other','redemption_other', 'redemption_handgun','redemption_long_gun','returned_other','rentals_handgun','rentals_long_gun','private_sale_handgun','private_sale_long_gun','private_sale_other','return_to_seller_handgun','return_to_seller_long_gun','return_to_seller_other','returned_handgun','returned_long_gun'], axis=1, inplace=True)

df_guns_q1.info() # check columns are dropped

In [ ]:
# figure displays overall trend of gun permits over the FBI guns time period 
sns.set_theme(style="darkgrid")

# calculate sum of totals for every onth
guns_overtime = df_guns_q1.groupby(['month'])['totals'].sum()


overtime_fig=sns.lineplot(data=guns_overtime, palette="crest");
overtime_fig.set_title('Overall Trend of Gun Permits')

The line chart displays the month on the x-axis and totals column on the y-axis. Gun registrations remained consistent in volume for several years until 2008 through 2010 when registrations started to slightly increase and trend at a higher volume with some notable upticks in volume between 2012 and 2014 and again in 2016 to 2017. 

In [ ]:
# calculating total handgun and long gun sum
handgun = df_guns_q1['handgun'].sum()
long_gun = df_guns_q1['long_gun'].sum()
handgun,long_gun # long gun permits were more prevalent in dataset

Comparing hand gun and long gun types shows that long guns accounted for a higher proportion of registrations nationwide

https://pythonspot.com/matplotlib-bar-chart/

In [ ]:
# create a list of gun types and the total sums 
guns = ['handgun','long_gun']
totals = [handgun , long_gun ]

# create a ndarray with the above gun type values 
gun_types = np.arange(len(guns))

# compare hand gun and long gun difference in a bar plot
plt.bar(gun_types, totals, align='center', alpha=0.5)
plt.xticks(gun_types, guns)
plt.ylabel('Gun Registrations')
plt.xlabel('Gun Types')
plt.title('Gun Type Comparison')

plt.show()

The plotly bar chart uses the gun types (hand gun and long gun) on the x-axis while gun registration totals is shown on the y-axis. The figure shows long gun registrations overall made up a larger share of gun registraitons in the nation, with roughly a quarter more than handguns 

### Research Question 2: What state has the highest volume of gun registrations?

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

In [ ]:
# calculate totals by state and show top five by totals
df_guns_q1_totals = df_guns_q1.groupby(['state'],as_index=False).sum().sort_values(by='totals', ascending=False).head(5)
df_guns_q1_totals

the calculation depicts the top five states for overall gun registrations, showing kentucky as the state with the highest totals in the nation

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html

https://pythonbasics.org/seaborn-barplot/

In [ ]:
# display top five states by overall gun registration volumes in a bar plot
sns.barplot(x = "state", y = "totals", hue ="state", data = df_guns_q1_totals,
            palette = "Blues",
            capsize = 0.05,             
            saturation = 8,             
            errcolor = 'b', errwidth = 5,  
            ci = 'sd'   
            ).set_title('Top Five States with the Highest Volume of Gun Permits')

The barplot shows the top five states in the nation for overall gun registrations with the totals column on the y-axis and state column on the x-axis. The figure shows that California and Texas (both with comparable population sizes) having an almost even number of gun registration totals

### Research Question 3: What state had the highest per capita gun registrations in 2010?

In [ ]:
#Get all the 2010 data from FBI Guns
guns_2010 = df_guns[df_guns.month == '2010']

guns_2010.head()

In [ ]:
guns_2010.set_index('state',inplace=True,drop=True) # reset index to prevent out of order indices
guns_2010.head(5)

In [ ]:
df_us_census_2.info() # check for changes

In [ ]:
df_us_census_2.info()

In [ ]:
# create dataframe with 2010 population data column

df_us_census_2010 = df_us_census_2['2010_Population_Estimates']
df_us_census_2010.to_frame().head()


In [ ]:
guns_2010.index = [x.title() for x in guns_2010.index] #capitalize the titles in the guns data to match census data

guns_2010.head() #check top few rows worked

In [ ]:
df_us_census_2010.to_frame().join(guns_2010) # join FBI guns and 2010 census data

In [ ]:
df_us_census_2010 = df_us_census_2010.to_frame().join(guns_2010) #join 2010 census and guns data and name with a new df

# df_us_census_2010.info() #check join worked

In [ ]:
# calculate per capita gun registrations in 2010 
percapita_2010 = df_us_census_2010['totals']/df_us_census_2010['2010_Population_Estimates']

percapita_2010_ = percapita_2010.sort_values(ascending=False).head(10)

percapita_2010_

Kentucky had the highest per capita gun registrations in 2010, with more than double the number of registrations that Utah had, the state with the second highest per capita amount

In [ ]:
# create a plot with the top 10 states for 2010 per capita gun registrations
percapita_2010_.plot(x='state', y='totals',
                        kind='bar', color=['orange']);
plt.title("Per Capita Gun Registrations (2010)",fontsize=15)
plt.ylabel('Per Capita Gun Registrations Count', fontsize=12)
plt.xlabel('Top Ten States Per Capita',fontsize=12)
plt.legend

The barplot shows the totals (gun registrations) column on the y-axis and the state column on the x-axis. Kentucy had a disproportionately higher per capita gun registration count in 2010 compared to any other state in the top ten list

### Research Question 4: What state had the highest per capita gun registrations in 2016?

In [ ]:
#Get all the 2016 data
guns_2016 = df_guns[df_guns.month == '2016']

guns_2016.head()

In [ ]:
guns_2016.set_index('state',inplace=True,drop=True) # reset indices incase out of order

In [ ]:
# create df with 2016 census population
df_us_census_2016 = df_us_census_2['2016_Population_Estimates']
df_us_census_2016.to_frame().head()

https://knowledge.udacity.com/questions/574175

In [ ]:
guns_2016.index = [x.title() for x in guns_2016.index] #capitalize state to match with census state column

In [ ]:
df_us_census_2016.to_frame().join(guns_2016).head() #join funs and 2016 data

In [ ]:
df_us_census_2016 = df_us_census_2016.to_frame().join(guns_2016).head() #create new df name with 2016 census and guns data

# df_us_census_2016.info()

In [ ]:
# calculate 2016 per capita 
percapita_2016 = df_us_census_2016['totals']/df_us_census_2016['2016_Population_Estimates']

percap_top_10_2016 = percapita_2016.sort_values(ascending=False).head(10)

percap_top_10_2016

Consistent with 2010 analysis, Kentucky remained the top state for per capita gun registrations in 2016. The state continued to account for double the amount of registrations although in this year, the second top state (Indiana) had a larger share of per capita registrations than Utah did (the second highest state for per capita registrations in 2010). In general the top states had more gun registrations in 2016 than 2010, indicating an overall upwards trend in gun registrations over time 

The figure depicts top ten states by per capita gun registrations, showing the state of Kentucky with more than double the per capita gun registration totals than any other state

In [ ]:
# display plot figure with the top 2016 per capita states
percap_top_10_2016.plot(x='state', y='totals',
                        kind='bar',
                        title="Per Capita Gun Registrations (2016)", color=['green']);
plt.title("Per Capita Gun Registrations (2016)",fontsize=15)
plt.ylabel('Per Capita Gun Registrations Count', fontsize=12)
plt.xlabel('Top Ten States Per Capita',fontsize=12)
plt.legend

The barplot shows states on the x-axis and per capita gun registration on the y-axis based on the above per capita calculation. Kentucky again is seen with the highest count per capita, followed by indiana. The remaining states all had a relatively even amount of gun registrations in 2016

<a id='conclusions'></a>
## Conclusions


Overall, the FBI Guns dataset showed gun permits increased over time, from 1998 through 2017. There were two noticeable spikes observed towards the end of the period, between 2013 and 2014 and again in 2016 to 2017. When comparing the two largest gun types (handguns and long guns), long guns accounted for 23 million more gun registrations.

According to the FBI Guns data, Kentucky accounted for the highest volume of total gun permits in the nation and had more than double the per capita gun registrations of any other state in both 2010 and 2016. The top ten states for gun registrations per capita generally remained the same in 2010 and 2016 with the exception of three states. 

Limitations: The US Census Data did not include several states/territories and only contained population data for 2010 and 2016 which limited the per capita analysis. 

The FBI Guns data had a high percentage of missing values which limited what you could use for the analysis. Additionally, 
sources indicated that there are varying state laws that impact the quality of data in the NICS dataset.

source: https://github.com/BuzzFeedNews/nics-firearm-background-checks
